In [58]:
import pandas as pd
from scipy.io import loadmat
import numpy as np

import h5py

In [29]:
#data_path_original = "Brown-Dwarfs/dataset_bd_wnames.csv"
mat_file = "original_dataset/ReadBrownDwarf.mat"

f = h5py.File(mat_file, 'r')
f.keys()


<KeysViewHDF5 ['#refs#', '#subsystem#', 'MCC', 'Perf', 'Precision', 'Recall', 'ReferenceResult', 'Result', 'a', 'ans', 'b', 'confMatrix', 'data', 'feat', 'fid', 'fold', 'idTE', 'idTR', 'indices', 'label', 'labelTE', 'labelTR', 'pat', 'pattern', 'rocObj', 'scoreLibSVMnoRed', 'test', 'tokens', 'train']>

In [133]:
idTR_list = []
with h5py.File(mat_file, 'r') as f:
    for i in range(5):
        idTR = f['idTR']
        ref = idTR[i,0]
        element = f[ref]
        df_label_TR = pd.DataFrame(idTR)
        element_array = np.array(element)

    #Extracting the data from the array
    for element in element_array:
        #One dataset has a bug in it, so we need to remove the last element
        if i == 1:
            dropout = int(element[4535])
            element = element[:4535]
            print("Dropout index = ",dropout)
        idTR_list.append(element.astype(int))
        # print(len(element))

idTR = np.array(idTR_list)
print(idTR.shape)

4535
Dropout index =  5669
4535
4535
4535
4535
(5, 4535)


In [132]:
idTE_list = []
with h5py.File(mat_file, 'r') as f:
    for i in range(5):
        idTE = f['idTE']
        ref = idTE[i,0]
        element = f[ref]
        df_label_TR = pd.DataFrame(idTE)
        element_array = np.array(element)
    #One dataset has a bug in it, so we need to add the dropout index to the array
    if i == 1:
        element_array = np.append(element_array, dropout)
        idTE_list.append(element_array.astype(int))
    else:
        idTE_list.append(element_array[0].astype(int))
    # print(len(element_array[0]))

idTE = np.array(idTE_list)
print(idTE.shape)

(5, 1134)


In [163]:
with h5py.File(mat_file, 'r') as f:
    labelTR_list = []
    for i in range(5):
        # Access the dataset
        data = f['labelTR']

        # Access a specific reference
        ref = data[i, 0]  # Change the indices to access different references

        # Dereference the reference
        element = f[ref]
        element_array = np.array(element)
        df_label_tr = pd.DataFrame(element_array)

        # Use numpy's vectorized operations to create true_labels
        true_labels = np.where(element_array == 2.0, 1, 0)
        #One dataset has a bug in it, so we need to remove the last element
        if i == 1:
            dropout_label = true_labels[4535]
            true_labels = true_labels[:4535]
        true_labels = true_labels.flatten()
        labelTR_list.append(true_labels)

labelTR = np.array(labelTR_list)
labelTR.shape

(5, 4535)

In [162]:
with h5py.File(mat_file, 'r') as f:
    labelTE_list = []
    for i in range(5):
        # Access the dataset
        data = f['labelTE']

        # Access a specific reference
        ref = data[i, 0]  # Change the indices to access different references

        # Dereference the reference
        element = f[ref]
        element_array = np.array(element)

        # Use numpy's vectorized operations to create true_labels
        true_labels = np.where(element_array == 2.0, 1, 0)
        #One dataset has a bug in it, so we need to remove the last element
        if i == 1:
            true_labels = np.append(true_labels, dropout_label)
        true_labels = true_labels.flatten()
        labelTE_list.append(true_labels)

labelTE = np.array(labelTE_list)
labelTE.shape

(5, 1134)

In [93]:
df_data.to_csv('original_dataset/matlab_file_sorted.csv', index=False)

Old data visulaized.

Read the data and find the different vectors

In [94]:
# data_path = ".\original_dataset\dataset_bd_wnames.csv" #Windows
data_path = "original_dataset/dataset_bd_wnames.csv"  #MACos
df = pd.read_csv(data_path)
df.head()
df.iloc[1,:]

ID           HD 203977
sp_type             A0
RAdeg        321.20044
DEdeg        25.880459
PS1gmag            NaN
e_PS1gmag          NaN
PS1rmag         7.6116
e_PS1rmag        0.001
PS1imag            NaN
e_PS1imag          NaN
PS1zmag         7.5167
e_PS1zmag        0.001
PS1ymag        13.5874
e_PS1ymag       0.0028
W1mag             6.88
W2mag            6.919
W3mag            6.946
W4mag            6.484
e_W1mag          0.059
e_W2mag           0.02
e_W3mag          0.016
e_W4mag          0.059
Jmag             6.896
Hmag             6.921
Kmag             6.932
e_Jmag           0.026
e_Hmag           0.024
e_Kmag           0.016
label                0
Name: 1, dtype: object

Fill missing values with the mean of the vector

In [95]:
for column in df.columns:
    if column in ["ID", "sp_type"]: #Skipping these
        continue
  
    df[column] = df[column].fillna(df[column].mean()) #Imputing NA values with mean

df.head()

,ID,sp_type,RAdeg,DEdeg,PS1gmag,e_PS1gmag,PS1rmag,e_PS1rmag,PS1imag,e_PS1imag,...,e_W2mag,e_W3mag,e_W4mag,Jmag,Hmag,Kmag,e_Jmag,e_Hmag,e_Kmag,label
0,* 11 Peg,A0V,326.80817,2.686124,5.55600,0.02303,5.760000,0.027126,5.947000,0.022877,...,0.050,0.015,0.044000,5.562,5.539,5.479,0.017,0.026,0.020,0
1,HD 203977,A0,321.20044,25.880459,15.12594,0.02303,7.611600,0.001000,14.918063,0.022877,...,0.020,0.016,0.059000,6.896,6.921,6.932,0.026,0.024,0.016,0
2,HD 83162,A0III/IV,144.03845,-12.459262,10.51200,0.02303,10.692000,0.027126,10.866000,0.022877,...,0.020,0.074,0.099197,10.408,10.400,10.378,0.026,0.021,0.023,0
3,* 10 Boo,A0Vs,209.66217,21.696203,15.12594,0.02303,14.982256,0.027126,6.860300,0.104200,...,0.046,0.015,0.037000,5.671,5.717,5.704,0.023,0.063,0.020,0
4,HD 27616,A0V,65.16255,-20.639620,15.12594,0.02303,14.982256,0.027126,6.320600,0.000000,...,0.072,0.015,0.036000,5.430,5.402,5.333,0.054,0.036,0.017,0


In [ ]:
index_Te = df["idTE"].astype(int)


Encode the vector "sp_type" as it is categorical

In [96]:
X = df.drop('ID', axis=1)
unique_values = len(X["sp_type"].unique()) # 504
X['sp_type'], _ = pd.factorize(X['sp_type'])

Split the dataframe/array into data and ground truth

In [97]:
y = X["label"]
X = X.drop(columns=["label"])

Using the actual dataset from the matlab file given instead. Overwriting the previous values of data_path, X, y. 

In [98]:
data_path = "original_dataset/matlab_file_sorted.csv"  #MACos
df = pd.read_csv(data_path)
y = df["label"]
X = df.drop(columns=["label"])

In [99]:
X.shape,y.shape

((5669, 26), (5669,))